# Code for running inference

10-18-2024

To convert this to a script, should have a name for the whole training task which goes in dataset.json and/or datalist.json because this will help for the task.json needed for inference.

Look into the minimal amount of metaata monai requires in the form of a task.json or other struct to initialize the builder and then run the ensemble


In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform

import mri_data
import monai_training

from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

from reload_recursive import reload_recursive

In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, filter_first_ses  # noqa: E402, F401
from monai_training import preprocess  # noqa: E402


Setup logging


In [3]:
log_dir = ".logs"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logger.add(
    os.path.join(log_dir, "file_{time:%Y_%m_%d}.log"), rotation="6h", level="DEBUG"
)

1

**Edit These**


In [4]:
#! Set these variables
work_dir_name = "choroid_pineal_pituitary2"
train_dataset_file_name = "training-dataset.json"
prediction_postfix = "choroid_pineal_pituitary2_pred"
task_name = "infer_choroid_pineal_pituitary"
modalities = ["flair", "t1"]

Set paths


In [5]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name
save_dir = drive_root / "3Tpioneer_bids_predictions"

prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)
print(prediction_filename)

taskfile_name = "inference-task.json"

flair.t1_choroid_pineal_pituitary2_pred.nii.gz


In [6]:
if not train_dataset_file.is_file():
    dataset = preprocess.parse_datalist(work_dir / "training-datalist.json", dataroot)
    preprocess.save_dataset(dataset, train_dataset_file)

In [7]:
def inference_exists(dataset: DataSet) -> DataSet:
    count = 0
    dataset_new = DataSet(dataset.dataroot)
    for scan in dataset:
        if not (save_dir / scan.relative_path / prediction_filename).is_file():
            dataset_new.append(scan)
        else:
            count += 1
    logger.info(f"{count} scans already have inference")
    return dataset_new

### Do Preparation

Get all the scans that were not in the training and testing set to create the inference dataset


In [8]:
# the scans that were used in the training
dataset_train, _ = preprocess.load_dataset(train_dataset_file)
dataset_train.dataroot = dataroot

# dataset_train2 has the same subject/sessions that are in dataset_train but with a subset of the keys
#   so that they can be compared to scans in the full data set when getting the set difference
dataset_proc = preprocess.DataSetProcesser.new_dataset(
    dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses, inference_exists]
)
dataset_full = dataset_proc.dataset
dataset_train2 = DataSet.dataset_like(dataset_train, ["subid", "sesid"])
dataset_inference = DataSet.from_scans(set(dataset_full) - set(dataset_train2))

2024-11-18 15:30:28.603 | INFO     | __main__:inference_exists:9 - 20 scans already have inference


Prepare the images in the inference dataset (i.e combine flair and t1 images)


In [9]:
dataset_proc = preprocess.DataSetProcesser(dataset_inference)
dataset_proc.prepare_images(["flair", "t1"])
dataset_proc.dataset.sort(key=lambda s: s.subid)

2024-11-18 15:30:28.613 | INFO     | monai_training.preprocess:prepare_images:107 - Prepare Images
 43%|████▎     | 226/526 [00:00<00:00, 444.14it/s]2024-11-18 15:30:29.125 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2106, sesid=20170205)
2024-11-18 15:30:29.174 | INFO     | mri_data.utils:merge_images:29 - fslmerge -a /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.t1.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
2024-11-18 15:30:30.848 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='1196', sesid='20161004', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids'), _root=PosixPath('/media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004'), image=None, label=None, cond=None, id=24112560784)>
2024-11-18 15:30:30.854 | WARNING  | monai_training

Image Exception : #22 :: Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated
/home/srs-9/fsl/share/fsl/bin/fslmerge: line 2: 449806 Aborted                 (core dumped) /home/srs-9/fsl/bin/fslmerge "$@"


 60%|██████    | 316/526 [00:02<00:02, 92.62it/s]2024-11-18 15:30:31.042 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1364, sesid=20170705)
2024-11-18 15:30:31.065 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1379, sesid=20161022)
 77%|███████▋  | 404/526 [00:02<00:00, 158.04it/s]2024-11-18 15:30:31.260 | INFO     | mri_data.utils:merge_images:29 - fslmerge -a /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.t1.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/t1.nii.gz
2024-11-18 15:30:31.689 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='2120', sesid='20170920', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids'), _root=PosixPath('/media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920'), image=Non

Image Exception : #22 :: Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated
/home/srs-9/fsl/share/fsl/bin/fslmerge: line 2: 449837 Aborted                 (core dumped) /home/srs-9/fsl/bin/fslmerge "$@"


100%|██████████| 526/526 [00:03<00:00, 159.57it/s]


Save the datalist and task files


In [10]:
images = []
for scan in dataset_proc.dataset[:10]:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

logger.info(f"Will run inference on {len(images)} scans")

datalist = {"testing": images}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

task = {
    "name": task_name,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": str(work_dir / "datalist.json"),
    "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, taskfile_name)
with open(task_file, "w") as f:
    json.dump(task, f, indent=4)

2024-11-18 15:30:39.481 | INFO     | __main__:<module>:6 - Will run inference on 10 scans


In [11]:
input_cfg = task_file  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [12]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage",
    "output_dir": save_dir,
    "data_root_dir": dataroot,
    "output_postfix": prediction_postfix,
    "separate_folder": False,
}

pred = ensemble(pred_param={"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

2024-11-18 15:31:55,084 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms1030/ses-20200304/flair.t1_choroid_pineal_pituitary2_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:  10%|█         | 1/10 [01:01<09:12, 61.42s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprec

2024-11-18 15:32:57,787 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms1031/ses-20191116/flair.t1_choroid_pineal_pituitary2_pred.nii.gz


Ensembling (rank 0)...:  20%|██        | 2/10 [02:04<08:17, 62.18s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.au

...

Ensembling (rank 0)...:  80%|████████  | 8/10 [08:10<01:58, 59.12s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.au

2024-11-18 15:40:05,739 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms1045/ses-20170418/flair.t1_choroid_pineal_pituitary2_pred.nii.gz


Ensembling (rank 0)...:  90%|█████████ | 9/10 [09:12<00:59, 60.00s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.au

2024-11-18 15:41:07,268 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms1046/ses-20181109/flair.t1_choroid_pineal_pituitary2_pred.nii.gz


Ensembling (rank 0)...: 100%|██████████| 10/10 [10:13<00:00, 61.36s/it]
